In [1]:
# https://2markdown.com
md_api_key = "YOUR_KEY"
# https://openai.com
openai_api_key = "YOUR_KEY"

In [3]:
import openai
import json
import requests

In [4]:
openai.api_key = openai_api_key

Let's build a function that can retrieve website content via 2markdown.com

In [5]:
def get_website_content(url):
    res = requests.post("https://2markdown.com/api/2md", json={"url":url}, headers={"X-Api-Key":md_api_key})
    
    if res.status_code == 200:
        return res.json()['article']
    
    return None

Which can look like this:

In [6]:
get_website_content("https://newsletterify.com")

"## Empower Your Knowledge,  Unclutter Your Inbox\n\n## Drowning in a Sea of Newsletters?\n\nAre you tired of wading through countless newsletters, struggling to find the\ninformation you need? Is your inbox cluttered with irrelevant content, making it\ndifficult to stay up-to-date in today's fast-paced world? You're not alone. We\nknow the feeling and that's why we created Newsletterify.\n\n\n![list of digests](https://newsletterify.com/newsletters.png)\n\n![list of digests](https://newsletterify.com/list_of_digests.png)\n\n## Introducing Newsletterify\n\nNewsletterify is your ultimate newsletter reader, designed to give you a\ndedicated email address for all your newsletter subscriptions. Organize,\ncustomize, and schedule digests based on topics and preferences, giving you the\nfreedom to gain a knowledge advantage on your terms.\n\n\n## Simplify Your Learning Journey with 3 Easy Steps\n\n- 1\\. Get your own email address for newsletters 📩\n- 2\\. Sign up for some newsletters✨\n- 3\

Now, let's glue this together with the OpenAPI API: [[source](https://platform.openai.com/docs/guides/gpt/function-calling)]

In [7]:
def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's https://newsletterify.com about?"}]
    functions = [
        {
            "name": "get_website_content",
            "description": "Retrieve the content of any website in URL format",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "The URL of the website, i.e. https://openai.com",
                    },
                },
                "required": ["url"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_website_content": get_website_content,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            url=function_args.get("url"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response


In [8]:
print(run_conversation())

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Newsletterify is a platform that aims to simplify the process of managing and reading newsletters. It provides users with a dedicated email address for their newsletter subscriptions, allowing them to organize, customize, and schedule digests based on their preferences. By using Newsletterify, users can unclutter their inbox, stay up-to-date with industry insights, and gain access to a world of knowledge without feeling overwhelmed. The platform also offers features such as an immersive reader, newsletter summarizations, and personal recommendations.",
        "role": "assistant"
      }
    }
  ],
  "created": 1687181778,
  "id": "chatcmpl-7T9LGDWj7ipyvpfVO5cyY19CDqhZA",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 98,
    "prompt_tokens": 561,
    "total_tokens": 659
  }
}
